# PyTorch CUDA 테스트 노트북
이 노트북은 PyTorch와 CUDA가 제대로 설정되었는지 확인하기 위한 기본 테스트를 포함하고 있습니다.

## 1. PyTorch 및 CUDA 버전 확인
PyTorch와 CUDA 버전이 올바르게 설정되었는지 확인합니다.

In [31]:
import torch
import platform
from datetime import datetime

print(f"현재 날짜 및 시간: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")
print(f"운영체제: {platform.platform()}")
print(f"Python 버전: {platform.python_version()}")
print(f"PyTorch 버전: {torch.__version__}")
print(f"CUDA 사용 가능: {torch.cuda.is_available()}")
print(f"CUDA 버전: {torch.version.cuda}")
print(f"cuDNN 버전: {torch.backends.cudnn.version() if torch.backends.cudnn.is_available() else '사용 불가'}")
print(f"사용 가능한 GPU 개수: {torch.cuda.device_count()}")

if torch.cuda.is_available():
    for i in range(torch.cuda.device_count()):
        print(f"GPU {i}: {torch.cuda.get_device_name(i)}")

현재 날짜 및 시간: 2025-04-08 04:40:23
운영체제: Linux-5.15.167.4-microsoft-standard-WSL2-x86_64-with-glibc2.35
Python 버전: 3.10.16
PyTorch 버전: 2.2.2
CUDA 사용 가능: True
CUDA 버전: 12.1
cuDNN 버전: 8902
사용 가능한 GPU 개수: 1
GPU 0: NVIDIA GeForce RTX 4060 Ti


## 2. 기본 CUDA 텐서 작업
CUDA를 사용하여 기본 텐서 작업을 수행합니다.

In [32]:
# CPU와 GPU 텐서 생성 및 이동
x = torch.rand(5, 3)
print(f"CPU 텐서:\n{x}\n")

if torch.cuda.is_available():
    # GPU에서 텐서 생성
    x_gpu = torch.rand(5, 3, device='cuda')
    print(f"GPU에서 생성된 텐서:\n{x_gpu}\n")
    
    # CPU 텐서를 GPU로 이동
    x_to_gpu = x.to('cuda')
    print(f"CPU에서 GPU로 이동된 텐서:\n{x_to_gpu}\n")
    
    # 텐서가 위치한 장치 확인
    print(f"x 텐서 장치: {x.device}")
    print(f"x_gpu 텐서 장치: {x_gpu.device}")
    print(f"x_to_gpu 텐서 장치: {x_to_gpu.device}")
else:
    print("CUDA가 사용 불가능합니다. CPU에서만 텐서 작업을 수행합니다.")

CPU 텐서:
tensor([[0.8993, 0.5611, 0.3627],
        [0.6949, 0.8376, 0.3835],
        [0.8757, 0.5431, 0.1674],
        [0.6420, 0.8558, 0.2687],
        [0.0568, 0.8261, 0.3783]])

GPU에서 생성된 텐서:
tensor([[0.9727, 0.0053, 0.9023],
        [0.0391, 0.3300, 0.3857],
        [0.2901, 0.1819, 0.5613],
        [0.9572, 0.0142, 0.1316],
        [0.6711, 0.4393, 0.3736]], device='cuda:0')

CPU에서 GPU로 이동된 텐서:
tensor([[0.8993, 0.5611, 0.3627],
        [0.6949, 0.8376, 0.3835],
        [0.8757, 0.5431, 0.1674],
        [0.6420, 0.8558, 0.2687],
        [0.0568, 0.8261, 0.3783]], device='cuda:0')

x 텐서 장치: cpu
x_gpu 텐서 장치: cuda:0
x_to_gpu 텐서 장치: cuda:0


## 3. 간단한 벤치마킹 - CPU vs GPU
CPU와 GPU에서 행렬 곱셈 연산 속도를 비교합니다.

In [46]:
import time

# 벤치마크 함수 정의
def benchmark_matmul(size, device='cpu'):
    """
    행렬 곱셈 연산 벤치마킹
    """
    a = torch.randn(size, size, device=device)
    b = torch.randn(size, size, device=device)
    
    if device == 'cuda':
        torch.cuda.synchronize()  # 정확한 타이밍을 위해 CUDA 작업이 완료될 때까지 대기
        
    start = time.time()
    
    # 행렬 곱셈 수행
    for _ in range(10):
        c = torch.matmul(a, b)
    
    if device == 'cuda':
        torch.cuda.synchronize()
    
    end = time.time()
    return end - start

# 다양한 크기로 벤치마크 실행
sizes = [128, 512, 1024, 2048]

print("CPU vs GPU 행렬 곱셈 벤치마크 (10회 반복 평균 시간)")
print("-" * 50)
print(f"{'크기':>10} | {'CPU (초)':>15} | {'GPU (초)':>15} | {'속도 향상':>15}")
print("-" * 50)

for size in sizes:
    cpu_time = benchmark_matmul(size, 'cpu')
    
    if torch.cuda.is_available():
        gpu_time = benchmark_matmul(size, 'cuda')
        speedup = cpu_time / gpu_time
    else:
        gpu_time = float('nan')
        speedup = float('nan')
    
    print(f"{size:10d} | {cpu_time:15.6f} | {gpu_time:15.6f} | {speedup:15.2f}x")

CPU vs GPU 행렬 곱셈 벤치마크 (10회 반복 평균 시간)
--------------------------------------------------
        크기 |         CPU (초) |         GPU (초) |           속도 향상
--------------------------------------------------
       128 |        0.001274 |        0.001630 |            0.78x
       512 |        0.005250 |        0.003285 |            1.60x
      1024 |        0.037085 |        0.024539 |            1.51x
      2048 |        0.238752 |        0.183460 |            1.30x


## 4. TensorRT 가용성 확인
TensorRT가 설치되어 있는지 확인합니다.

In [25]:
# TensorRT 설치 확인
try:
    import tensorrt as trt
    print(f"TensorRT 버전: {trt.__version__}")
    print("TensorRT가 성공적으로 설치되었습니다.")
except ImportError:
    print("TensorRT가 설치되지 않았거나 Python 환경에서 접근할 수 없습니다.")

# torch-tensorrt 확인 시도(선택 사항)
try:
    import torch_tensorrt
    print(f"torch-tensorrt 버전: {torch_tensorrt.__version__}")
    print("torch-tensorrt가 설치되었습니다.")
except ImportError:
    print("torch-tensorrt가 설치되어 있지 않습니다.")

TensorRT 임포트 오류: No module named 'tensorrt'
해결 방법: pip install tensorrt 또는 apt install nvidia-tensorrt 실행
torch-tensorrt 임포트 오류: No module named 'torch_tensorrt'
torch-tensorrt는 선택적으로 사용 가능합니다: pip install torch-tensorrt


## 5. 간단한 CNN 모델 훈련 예제
GPU를 활용한 간단한 CNN 모델 훈련 예제입니다.

In [ ]:
# 필요한 패키지 설치 확인 (필요한 경우 주석 해제)
!pip install transformers datasets evaluate accelerate

In [52]:
import torch
import time
from transformers import AutoTokenizer, AutoModelForSequenceClassification, TrainingArguments, Trainer
from datasets import load_dataset
import numpy as np
from datasets import load_metric
import random

# 장치 선택
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(f"훈련에 사용할 장치: {device}")

# 재현성을 위한 시드 설정
seed = 42
random.seed(seed)
np.random.seed(seed)
torch.manual_seed(seed)
if torch.cuda.is_available():
    torch.cuda.manual_seed_all(seed)

try:
    # 작은 텍스트 분류 데이터셋 로드 (SST-2)
    dataset = load_dataset("glue", "sst2", split=["train[:1000]", "validation[:200]"])
    train_dataset, eval_dataset = dataset[0], dataset[1]
    print(f"훈련 데이터셋 크기: {len(train_dataset)}")
    print(f"평가 데이터셋 크기: {len(eval_dataset)}")
    
    # 토크나이저 및 모델 로드
    model_name = "distilbert-base-uncased"  # 작은 모델 사용
    tokenizer = AutoTokenizer.from_pretrained(model_name)
    model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=2).to(device)
    
    # 데이터셋 전처리
    def preprocess_function(examples):
        return tokenizer(examples["sentence"], truncation=True, padding="max_length", max_length=128)
    
    encoded_train_dataset = train_dataset.map(preprocess_function, batched=True)
    encoded_eval_dataset = eval_dataset.map(preprocess_function, batched=True)
    
    # 평가 메트릭 정의
    def compute_metrics(eval_pred):
        predictions, labels = eval_pred
        predictions = np.argmax(predictions, axis=1)
        return {"accuracy": np.mean(predictions == labels)}
    
    # 훈련 인자 설정 
    batch_size = 16
    training_args = TrainingArguments(
        output_dir="./results",
        num_train_epochs=1,  # 테스트를 위한 1 에포크
        per_device_train_batch_size=batch_size,
        per_device_eval_batch_size=batch_size,
        warmup_steps=50,
        weight_decay=0.01,
        logging_dir="./logs",
        logging_steps=10,
        eval_steps=50,
        evaluation_strategy="steps",
        save_steps=1000,  # 저장 비활성화
        save_strategy="steps",
        load_best_model_at_end=False,  # 테스트를 위해 비활성화
        report_to="none",  # 리포팅 비활성화
    )
    
    # 트레이너 초기화
    trainer = Trainer(
        model=model,
        args=training_args,
        train_dataset=encoded_train_dataset,
        eval_dataset=encoded_eval_dataset,
        compute_metrics=compute_metrics,
        tokenizer=tokenizer,
    )
    
    # 시간 측정 시작
    print("\n학습 시작...")
    start_time = time.time()
    
    # 학습 실행 (예시 삼아 3 스텝만)
    max_steps = 3
    train_results = trainer.train(max_steps=max_steps)
    
    # 시간 측정 종료
    end_time = time.time()
    training_time = end_time - start_time
    
    # 평가 실행
    eval_results = trainer.evaluate()
    
    # 결과 출력
    print(f"\n{max_steps} 스텝 훈련 완료!")
    print(f"훈련 시간: {training_time:.2f}초 (스텝당 {training_time/max_steps:.2f}초)")
    print(f"평가 정확도: {eval_results['eval_accuracy']:.4f}")
    print(f"평가 손실: {eval_results['eval_loss']:.4f}")
    print(f"사용한 장치: {device}")
    
    # 모델 메모리 사용량 출력
    if torch.cuda.is_available():
        memory_allocated = torch.cuda.memory_allocated(0) / 1024**2
        memory_reserved = torch.cuda.memory_reserved(0) / 1024**2
        print(f"CUDA 메모리 할당: {memory_allocated:.2f} MB")
        print(f"CUDA 메모리 예약: {memory_reserved:.2f} MB")
    
except Exception as e:
    print(f"훈련 중 오류 발생: {e}")

ImportError: cannot import name 'load_metric' from 'datasets' (/opt/conda/envs/pytorch_env/lib/python3.10/site-packages/datasets/__init__.py)

## 6. Hugging Face Diffusion 모델을 활용한 이미지 생성 테스트
Hugging Face의 Diffusion 모델을 사용하여 GPU 성능을 테스트합니다.

In [ ]:
# 필요한 패키지 설치 확인 (필요한 경우 주석 해제)
# !pip install diffusers accelerate

In [49]:
import torch
import time
from diffusers import StableDiffusionPipeline
from IPython.display import display

# 장치 확인
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(f"이미지 생성에 사용할 장치: {device}")

try:
    if torch.cuda.is_available():
        # GPU 메모리 사용량을 줄이기 위한 설정
        model_id = "runwayml/stable-diffusion-v1-5"  # 가장 널리 사용되는 모델
        
        # 메모리 최적화 옵션과 함께 모델 로드
        torch_dtype = torch.float16 if torch.cuda.is_available() else torch.float32
        pipe = StableDiffusionPipeline.from_pretrained(
            model_id, 
            torch_dtype=torch_dtype,
            revision="fp16",
            use_safetensors=True
        )
        pipe = pipe.to(device)
        
        # 추가 메모리 최적화
        pipe.enable_attention_slicing()
        
        # 프롬프트 설정 및 이미지 생성
        prompt = "한국의 아름다운 야경 사진, 서울 남산타워"
        
        # 이미지 생성 시간 측정
        print("\n이미지 생성 시작...")
        start_time = time.time()
        
        # 이미지 생성
        with torch.autocast("cuda"):
            image = pipe(prompt, num_inference_steps=20, height=512, width=512).images[0]
        
        end_time = time.time()
        
        # 결과 표시
        print(f"이미지 생성 완료! 소요 시간: {end_time - start_time:.2f}초")
        display(image)
        
        # 메모리 사용량 출력
        memory_allocated = torch.cuda.memory_allocated(0) / 1024**3
        memory_reserved = torch.cuda.memory_reserved(0) / 1024**3
        print(f"CUDA 메모리 할당: {memory_allocated:.2f} GB")
        print(f"CUDA 메모리 예약: {memory_reserved:.2f} GB")
    else:
        print("CUDA를 사용할 수 없어 이미지 생성 테스트를 건너뜁니다.")
        print("이 테스트는 최소 10GB 이상의 VRAM이 필요합니다.")

except Exception as e:
    print(f"이미지 생성 중 오류 발생: {e}")
    print("메모리 부족 오류가 발생한 경우 다음을 시도해 보세요:")
    print("1. 더 작은 해상도 설정 (height=384, width=384)")
    print("2. 더 적은 inference_steps 설정 (예: 15)")
    print("3. 더 적은 VRAM을 사용하는 모델 선택 (예: CompVis/stable-diffusion-v1-4)")

이미지 생성에 사용할 장치: cuda:0


Couldn't connect to the Hub: 404 Client Error. (Request ID: Root=1-67f4a924-7bb53f7936e8a29a37120af0;9ff733fa-a744-4740-ad46-b15c6f102ac6)

Revision Not Found for url: https://huggingface.co/api/models/stable-diffusion-v1-5/stable-diffusion-v1-5/revision/fp16.
Invalid rev id: fp16.
Will try to load from local cache.


이미지 생성 중 오류 발생: Cannot load model runwayml/stable-diffusion-v1-5: model is not cached locally and an error occurred while trying to fetch metadata from the Hub. Please check out the root cause in the stacktrace above.
메모리 부족 오류가 발생한 경우 다음을 시도해 보세요:
1. 더 작은 해상도 설정 (height=384, width=384)
2. 더 적은 inference_steps 설정 (예: 15)
3. 더 적은 VRAM을 사용하는 모델 선택 (예: CompVis/stable-diffusion-v1-4)
